In [1]:
from sagemaker.model_monitor import DataCaptureConfig

# Please fill in the following for enabling data capture
s3_capture_upload_path = f"s3://sagemaker-us-west-2-846634201516/monitoring/"  # example: s3://bucket-name/path/to/endpoint-data-capture/


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [3]:
import sagemaker
import boto3
# SageMaker session
sess = sagemaker.Session()
# get session bucket name
bucket = sess.default_bucket()
# bucket prefix or the subfolder for everything we produce
prefix = "mlops"
# get sagemaker role
sagemaker_role = sagemaker.get_execution_role()
# s3 client
s3_client = boto3.client("s3")

region = boto3.Session().region_name
boto_session = boto3.Session(region_name=region)


sagemaker_client = boto_session.client(service_name="sagemaker", region_name=region)
sagemaker_session = sagemaker.session.Session(
    boto_session=boto_session, sagemaker_client=sagemaker_client
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [5]:
%%time
endpoint_name = "CustomerChurn"
predictor = sagemaker.predictor.Predictor(
    endpoint_name=endpoint_name, sagemaker_session=sagemaker_session
)

# Change parameters as you would like - adjust sampling percentage,
#  chose to capture request or response or both
data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=100,
    destination_s3_uri=s3_capture_upload_path,
    kms_key_id=None,
    capture_options=["REQUEST", "RESPONSE"],
    csv_content_types=["text/csv"],
    json_content_types=["application/json"],
)

# Now it is time to apply the new configuration and wait for it to be applied
predictor.update_data_capture_config(data_capture_config=data_capture_config)
sess.wait_for_endpoint(endpoint=endpoint_name)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
------!!CPU times: user 302 ms, sys: 16.6 ms, total: 318 ms
Wall time: 4min 2s


{'EndpointName': 'CustomerChurn',
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:846634201516:endpoint/CustomerChurn',
 'EndpointConfigName': 'CustomerChurn-2024-03-30-22-58-31-454',
 'ProductionVariants': [{'VariantName': 'AllTraffic',
   'DeployedImages': [{'SpecifiedImage': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost:latest',
     'ResolvedImage': '433757028032.dkr.ecr.us-west-2.amazonaws.com/xgboost@sha256:0c8f830ac408e6dee08445fb60392e9c3f05f790a4b3c07ec22327c08f75bcbf',
     'ResolutionTime': datetime.datetime(2024, 3, 30, 22, 58, 32, 917000, tzinfo=tzlocal())}],
   'CurrentWeight': 1.0,
   'DesiredWeight': 1.0,
   'CurrentInstanceCount': 1,
   'DesiredInstanceCount': 1}],
 'DataCaptureConfig': {'EnableCapture': True,
  'CaptureStatus': 'Started',
  'CurrentSamplingPercentage': 100,
  'DestinationS3Uri': 's3://sagemaker-us-west-2-846634201516/monitoring/'},
 'EndpointStatus': 'InService',
 'CreationTime': datetime.datetime(2024, 3, 30, 22, 54, 21, 707000, tzinfo=tzlocal()

In [6]:
##'s3://bucketname/path/to/baseline/data' - Where your validation data is
baseline_data_uri = f"s3://sagemaker-us-west-2-846634201516/ml_deploy/validation"
##'s3://bucketname/path/to/baseline/data' - Where the results are to be stored in
baseline_results_uri = f"s3://sagemaker-us-west-2-846634201516/monitoring/baseline/results"

print("Baseline data uri: {}".format(baseline_data_uri))
print("Baseline results uri: {}".format(baseline_results_uri))

Baseline data uri: s3://sagemaker-us-west-2-846634201516/ml_deploy/validation
Baseline results uri: s3://sagemaker-us-west-2-846634201516/monitoring/baseline/results


In [7]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker import get_execution_role
import datetime

role = get_execution_role(sagemaker_session=sess)
prefix = "customerchurn"

datetime_stamp = datetime.datetime.now().strftime("%Y-%m-%d-%H%M%S")

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=1800,
    base_job_name=f"{prefix}-monitor-{datetime_stamp}",
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [8]:
%%time

monitor_baseline = my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=False),
    output_s3_uri=baseline_results_uri,
    job_name=f"{prefix}-monitor-baseline-{datetime_stamp}",
    wait=True,
)

INFO:sagemaker:Creating processing-job with name customerchurn-monitor-baseline-2024-03-30-231131


.............................2024-03-30 23:16:25.847607: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-30 23:16:25.847644: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-30 23:16:28.201222: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-03-30 23:16:28.201251: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2024-03-30 23:16:28.201276: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-0-107-151.us-west-2.compute.internal): /proc/driver/nvidia/version does not exist
2024-

In [13]:
from time import gmtime, strftime
import boto3

client = boto3.client("sagemaker")


def get_last_processing_job():
    response = client.list_processing_jobs(
        NameContains=f"{prefix}-monitor-baseline-{datetime_stamp}",
        StatusEquals="Completed",
        SortBy="CreationTime",
        SortOrder="Descending",
        MaxResults=20,
    )
    pprint.pprint(response["ProcessingJobSummaries"][0])
    return response["ProcessingJobSummaries"][0]["ProcessingJobName"]

In [14]:
from sagemaker.processing import ProcessingJob
from sagemaker.estimator import Estimator
from sagemaker.model_monitor.model_monitoring import ModelMonitor
import pprint
my_default_monitor_name = get_last_processing_job()

{'CreationTime': datetime.datetime(2024, 3, 30, 23, 11, 34, 579000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 774000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 171000, tzinfo=tzlocal()),
 'ProcessingJobArn': 'arn:aws:sagemaker:us-west-2:846634201516:processing-job/customerchurn-monitor-baseline-2024-03-30-231131',
 'ProcessingJobName': 'customerchurn-monitor-baseline-2024-03-30-231131',
 'ProcessingJobStatus': 'Completed'}


In [15]:
my_default_monitor_reload = ProcessingJob.from_processing_name(sess, my_default_monitor_name)

response = client.describe_processing_job(ProcessingJobName=my_default_monitor_name)
pprint.pprint(response)

{'AppSpecification': {'ImageUri': '159807026194.dkr.ecr.us-west-2.amazonaws.com/sagemaker-model-monitor-analyzer'},
 'CreationTime': datetime.datetime(2024, 3, 30, 23, 11, 34, 579000, tzinfo=tzlocal()),
 'Environment': {'dataset_format': '{"csv": {"header": false, '
                                   '"output_columns_position": "START"}}',
                 'dataset_source': '/opt/ml/processing/input/baseline_dataset_input',
                 'output_path': '/opt/ml/processing/output',
                 'publish_cloudwatch_metrics': 'Disabled'},
 'ExitMessage': 'Completed: Job completed successfully with no violations.',
 'LastModifiedTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 774000, tzinfo=tzlocal()),
 'ProcessingEndTime': datetime.datetime(2024, 3, 30, 23, 18, 25, 171000, tzinfo=tzlocal()),
 'ProcessingInputs': [{'AppManaged': False,
                       'InputName': 'baseline_dataset_input',
                       'S3Input': {'LocalPath': '/opt/ml/processing/input/baseline_da

In [16]:
import pandas as pd

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

,name,inferred_type,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data
0,_c0,Fractional,1038,0,0.503854,523.000000,0.499985,0.000000,1.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,_c1,Fractional,1038,0,102.279383,106166.000000,57.590835,1.000000,200.000000,"[{'lower_bound': 1.0, 'upper_bound': 20.9, 'co...",0.64,2048.0,"[[1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 3.0, 3.0,..."
2,_c2,Fractional,1038,0,231.406551,240200.000000,272.249153,0.000000,1300.000000,"[{'lower_bound': 0.0, 'upper_bound': 130.0, 'c...",0.64,2048.0,"[[0.0, 0.0, 500.0, 700.0, 0.0, 0.0, 0.0, 500.0..."
3,_c3,Fractional,1038,0,5.578486,5790.468231,3.381909,0.014247,16.504147,"[{'lower_bound': 0.014247142810832612, 'upper_...",0.64,2048.0,"[[1.442509561551946, 4.93641897645424, 7.03572..."
4,_c4,Fractional,1038,0,3.443160,3574.000000,1.710825,0.000000,8.000000,"[{'lower_bound': 0.0, 'upper_bound': 0.8, 'cou...",0.64,2048.0,"[[3.0, 3.0, 5.0, 4.0, 3.0, 2.0, 5.0, 6.0, 3.0,..."
5,_c5,Fractional,1038,0,5.102953,5296.865235,2.186658,0.014361,11.233955,"[{'lower_bound': 0.014361348439859256, 'upper_...",0.64,2048.0,"[[2.2467101421396123, 0.9736083883577038, 3.13..."
6,_c6,Fractional,1038,0,3.103083,3221.000000,2.515074,0.000000,13.000000,"[{'lower_bound': 0.0, 'upper_bound': 1.3, 'cou...",0.64,2048.0,"[[1.0, 2.0, 5.0, 0.0, 5.0, 1.0, 8.0, 2.0, 1.0,..."
7,_c7,Fractional,1038,0,4.014886,4167.451342,1.603892,0.077795,10.183378,"[{'lower_bound': 0.07779530526817169, 'upper_b...",0.64,2048.0,"[[5.052209883015463, 4.758515666071381, 5.1068..."
8,_c8,Fractional,1038,0,228.275530,236950.000000,95.956864,0.000000,500.000000,"[{'lower_bound': 0.0, 'upper_bound': 50.0, 'co...",0.64,2048.0,"[[150.0, 150.0, 200.0, 250.0, 400.0, 350.0, 45..."
9,_c9,Fractional,1038,0,5.027020,5218.047012,1.015671,2.223449,8.181336,"[{'lower_bound': 2.22344893121115, 'upper_boun...",0.64,2048.0,"[[4.227688847116808, 4.350906105573603, 5.5186..."


In [17]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative
0,_c0,Fractional,1.0,True
1,_c1,Fractional,1.0,True
2,_c2,Fractional,1.0,True
3,_c3,Fractional,1.0,True
4,_c4,Fractional,1.0,True
5,_c5,Fractional,1.0,True
6,_c6,Fractional,1.0,True
7,_c7,Fractional,1.0,True
8,_c8,Fractional,1.0,True
9,_c9,Fractional,1.0,True


In [11]:
sagemaker_runtime = boto3.client(
    "sagemaker-runtime", region_name='us-west-2')

response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Body = "2.0,400.0,0.38571846040122537,2.0,4.177940384158745,0.0,3.745462710628048,250.0,3.699591756294294,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0"
)
print(response['Body'].read().decode('utf-8'))

0.9136815667152405


In [33]:
for x in range(20):
    response = sagemaker_runtime.invoke_endpoint(
    EndpointName=endpoint_name, 
    ContentType='text/csv',
    Body = "2.0,400.0,0.38571846040122537,2.0,4.177940384158745,0.0,3.745462710628048,250.0,3.699591756294294,1.0,11.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0"
)
    print(response['Body'].read().decode('utf-8'))

0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405
0.9136815667152405


In [30]:
import time

MODEL_MONITOR_SCHEDULE_NAME = "mlops-data-quality-schedule"
sm_client = boto3.client("sagemaker")
monitoring_executions = sm_client.list_monitoring_executions(
    MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
)

# Check if monitoring job has started
while not (monitoring_executions.get("MonitoringExecutionSummaries")):

    # Progress update
    print("Waiting for the data drift model monitoring processing job to start...")

    # Pause for 60 seconds
    time.sleep(60)

    # Get the first monitoring execution details
    monitoring_executions = sm_client.list_monitoring_executions(
        MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
    )

print("The data drift model monitoring processing job has started.")

Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waiting for the data drift model monitoring processing job to start...
Waitin

In [41]:
mon_executions = my_default_monitor.list_executions()
print("We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.\nWe will have to wait till we hit the hour...")

while len(mon_executions) == 0:
    print("Waiting for the 1st execution to happen...")
    time.sleep(60)
    mon_executions = my_default_monitor.list_executions()

We created a hourly schedule above and it will kick off executions ON the hour (plus 0 - 20 min buffer.
We will have to wait till we hit the hour...


In [42]:
latest_execution = mon_executions[-1] # latest execution's index is -1, previous is -2 and so on..


print("Latest execution status: {}".format(latest_execution.describe()['ProcessingJobStatus']))
print("Latest execution result: {}".format(latest_execution.describe()['ExitMessage']))

latest_job = latest_execution.describe()
if (latest_job['ProcessingJobStatus'] != 'Completed'):
        print("====STOP==== \n No completed executions to inspect further. Please wait till an execution completes or investigate previously reported failures.")



Latest execution status: Completed
Latest execution result: Completed: Job completed successfully with no violations.


In [43]:
report_uri=latest_execution.output.destination
print('Report Uri: {}'.format(report_uri))

Report Uri: s3://sagemaker-us-west-2-846634201516/reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00


In [44]:
from urllib.parse import urlparse
s3uri = urlparse(report_uri)
report_bucket = s3uri.netloc
report_key = s3uri.path.lstrip('/')
print('Report bucket: {}'.format(report_bucket))
print('Report key: {}'.format(report_key))

s3_client = boto3.Session().client('s3')
result = s3_client.list_objects(Bucket=report_bucket, Prefix=report_key)
report_files = [report_file.get("Key") for report_file in result.get('Contents')]
print("Found Report Files:")
print("\n ".join(report_files))

Report bucket: sagemaker-us-west-2-846634201516
Report key: reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00
Found Report Files:
reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/constraint_violations.json
 reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/constraints.json
 reports//CustomerChurn/mlops-data-quality-schedule/2024/03/31/00/statistics.json


In [6]:
import boto3
sm_client = boto3.client("sagemaker")
MODEL_MONITOR_SCHEDULE_NAME = "mlops-data-quality-schedule"

response = sm_client.delete_monitoring_schedule(
    MonitoringScheduleName=MODEL_MONITOR_SCHEDULE_NAME
)


In [7]:
endpoint_name = "CustomerChurn"
sm_client.delete_endpoint(EndpointName=endpoint_name)

{'ResponseMetadata': {'RequestId': 'c9939153-4670-4f91-a482-bdafbe4886e1',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'c9939153-4670-4f91-a482-bdafbe4886e1',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 31 Mar 2024 03:29:38 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}